# Example usage for analysis pipeline

In this notebook, we will be exploring the functionality that allows to run various classification and resampling methods of different datasets using both Python code and the command line interface (CLI). We will be able to compare the results and efficiency of these methods in order to determine the best approach for our specific use case.

## 1. Python code

### Imports

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.metrics import geometric_mean_score
from pathlib import Path
from sklearn.metrics import accuracy_score
from tempfile import NamedTemporaryFile
from scikit_posthocs import posthoc_wilcoxon

from multi_imbalance.datasets.analysis import AnalysisPipeline, Config, Result
from multi_imbalance.datasets import load_datasets
from multi_imbalance.resampling.soup import SOUP
from multi_imbalance.resampling.spider import SPIDER3
from multi_imbalance.resampling.static_smote import StaticSMOTE
from multi_imbalance.resampling.global_cs import GlobalCS
from multi_imbalance.resampling.mdo import MDO

Load datasets from `data.tar.gz` file to csv files

In [3]:
_ = load_datasets(save_to_csv=True)

### Prepare configuration for analysis pipeline

In this example we will be working with three datasets: glass, new_ecoli, and dermatology. We will be applying two classifiers (Decision Tree, K-Nearest Neighbors) with two different configurations each to these datasets. We will be using three resampling methods (GlobalCS, MDO, SOUP) with default configurations as well as special configurations for MDO and glass dataset. These combinations will be evaluated using two metrics: geometric mean score and accuracy score. We will be repeating each combination five times and using the train_test_split method to divide the data into train and test sets.

In [18]:
cwd = Path.cwd()

config = {
    "datasets": [cwd.parents[1] / "data" / "csv" / "glass.csv", cwd.parents[1] / "data" / "csv" / "new_ecoli.csv", cwd.parents[1] / "data" / "csv" / "dermatology.csv"],
    "classifiers": {
        DecisionTreeClassifier: [{"max_depth" : 100}, {}],
        KNeighborsClassifier: [{"n_neighbors": 7}, {}],
    },
    "resampling_methods": {
        GlobalCS: {"default": {"shuffle": True}},
        MDO: {"default": {"k1_frac": 0.3, "maj_int_min":{"maj": [0, 1], "min": [2, 3, 4, 5]}}, "glass": {"k1_frac": 0.5}},
        SOUP: {"default" : {"shuffle": True}},
    },
    "metrics": {geometric_mean_score: {"correction": 0.001}, accuracy_score: {}},
    "n_repeats": 10, 
    "split_method": ["train_test",{}],
}

Prepare temporary file for result

In [19]:
result_file = NamedTemporaryFile(suffix=".csv")
result_file.close()

Create an AnalysisPipeline object and run the analysis, including a comparison of training without resampling to demonstrate the effectiveness of the resampling methods.

In [20]:
c = Config.from_dict(config)
pipeline = AnalysisPipeline(c)
pipeline.run_analysis(result_file.name, train_without_resampling = True)

c:\Users\Mateusz\Desktop\venvy\.project_venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mateusz\Desktop\venvy\.project_venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mateusz\Desktop\venvy\.project_venv\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mateusz\Desktop\venvy\.project_venv\lib\site-pack

Generate summary for selected classifiers, metric and dataset. If you don't know which names should you write you can use appropriate property to find e.g. dataset name

In [21]:
pipeline.dataset_names, pipeline.clf_names

(['dermatology', 'glass', 'new_ecoli'],
 ['kneighborsclassifier', 'decisiontreeclassifier'])

In [29]:
query_dict = {"classifier": ["decisiontreeclassifier", "kneighborsclassifier"], "metric_name": ["geometric_mean_score"], "dataset_name": ["glass", "new_ecoli"]}

results = pipeline.generate_summary(query_dict,save_to_csv=False, csv_path=result_file.name, aggregate_func=[min])

There is two classifiers, one metric and two datasets, so the length of results will be $2\cdot1\cdot2=4$

In [30]:
len(results)

4

#### First result

In [31]:
results[0]

metric_value  \
                                                                                                      mean   
metric_name          classifier             dataset_name resampling_method clf_params                        
geometric_mean_score decisiontreeclassifier glass        Not defined       {'max_depth': 100}     0.356512   
                                                                           {}                     0.566907   
                                                         globalcs          {'max_depth': 100}     0.477872   
                                                                           {}                     0.447095   
                                                         mdo               {'max_depth': 100}     0.589195   
                                                                           {}                     0.505958   
                                                         soup              {'max_depth': 100}     0.613124   
                                                                           {}                     0.582646   

                                                                                                         \
                                                                                                    std   
metric_name          classifier             dataset_name resampling_method clf_params                     
geometric_mean_score decisiontreeclassifier glass        Not defined       {'max_depth': 100}  0.222512   
                                                                           {}                  0.185909   
                                                         globalcs          {'max_depth': 100}  0.217623   
                                                                           {}                  0.237363   
                                                         mdo               {'max_depth': 100}  0.170955   
                                                                           {}                  0.239950   
                                                         soup              {'max_depth': 100}  0.218905   
                                                                           {}                  0.198582   

                                                                                                         
                                                                                                    min  
metric_name          classifier             dataset_name resampling_method clf_params                    
geometric_mean_score decisiontreeclassifier glass        Not defined       {'max_depth': 100}  0.177231  
                                                                           {}                  0.225857  
                                                         globalcs          {'max_depth': 100}  0.179135  
                                                                           {}                  0.057638  
                                                         mdo               {'max_depth': 100}  0.205291  
                                                                           {}                  0.068824  
                                                         soup              {'max_depth': 100}  0.204827  
                                                                           {}                  0.209060

#### Second result

In [27]:
results[1]

metric_value  \
                                                                                                      mean   
metric_name          classifier             dataset_name resampling_method clf_params                        
geometric_mean_score decisiontreeclassifier new_ecoli    Not defined       {'max_depth': 100}     0.749784   
                                                                           {}                     0.698934   
                                                         globalcs          {'max_depth': 100}     0.717990   
                                                                           {}                     0.700889   
                                                         mdo               {'max_depth': 100}     0.767789   
                                                                           {}                     0.771318   
                                                         soup              {'max_depth': 100}     0.738233   
                                                                           {}                     0.714600   

                                                                                                         \
                                                                                                    std   
metric_name          classifier             dataset_name resampling_method clf_params                     
geometric_mean_score decisiontreeclassifier new_ecoli    Not defined       {'max_depth': 100}  0.066426   
                                                                           {}                  0.052891   
                                                         globalcs          {'max_depth': 100}  0.034846   
                                                                           {}                  0.080018   
                                                         mdo               {'max_depth': 100}  0.045542   
                                                                           {}                  0.048929   
                                                         soup              {'max_depth': 100}  0.053773   
                                                                           {}                  0.063548   

                                                                                                         
                                                                                                    min  
metric_name          classifier             dataset_name resampling_method clf_params                    
geometric_mean_score decisiontreeclassifier new_ecoli    Not defined       {'max_depth': 100}  0.602696  
                                                                           {}                  0.620988  
                                                         globalcs          {'max_depth': 100}  0.670200  
                                                                           {}                  0.565038  
                                                         mdo               {'max_depth': 100}  0.713386  
                                                                           {}                  0.703493  
                                                         soup              {'max_depth': 100}  0.649839  
                                                                           {}                  0.583691

#### Third result

In [33]:
results[2]

metric_value  \
                                                                                                    mean   
metric_name          classifier           dataset_name resampling_method clf_params                        
geometric_mean_score kneighborsclassifier glass        Not defined       {'n_neighbors': 7}     0.204697   
                                                                         {}                     0.171839   
                                                       globalcs          {'n_neighbors': 7}     0.543043   
                                                                         {}                     0.672387   
                                                       mdo               {'n_neighbors': 7}     0.387075   
                                                                         {}                     0.145610   
                                                       soup              {'n_neighbors': 7}     0.585421   
                                                                         {}                     0.354566   

                                                                                                       \
                                                                                                  std   
metric_name          classifier           dataset_name resampling_method clf_params                     
geometric_mean_score kneighborsclassifier glass        Not defined       {'n_neighbors': 7}  0.191663   
                                                                         {}                  0.126395   
                                                       globalcs          {'n_neighbors': 7}  0.177286   
                                                                         {}                  0.088290   
                                                       mdo               {'n_neighbors': 7}  0.206910   
                                                                         {}                  0.061634   
                                                       soup              {'n_neighbors': 7}  0.154868   
                                                                         {}                  0.206900   

                                                                                                       
                                                                                                  min  
metric_name          classifier           dataset_name resampling_method clf_params                    
geometric_mean_score kneighborsclassifier glass        Not defined       {'n_neighbors': 7}  0.068485  
                                                                         {}                  0.075119  
                                                       globalcs          {'n_neighbors': 7}  0.202031  
                                                                         {}                  0.536591  
                                                       mdo               {'n_neighbors': 7}  0.078940  
                                                                         {}                  0.074212  
                                                       soup              {'n_neighbors': 7}  0.206894  
                                                                         {}                  0.153585

#### Fourth result

In [34]:
results[3]

metric_value  \
                                                                                                    mean   
metric_name          classifier           dataset_name resampling_method clf_params                        
geometric_mean_score kneighborsclassifier new_ecoli    Not defined       {'n_neighbors': 7}     0.848028   
                                                                         {}                     0.829293   
                                                       globalcs          {'n_neighbors': 7}     0.779214   
                                                                         {}                     0.789073   
                                                       mdo               {'n_neighbors': 7}     0.784751   
                                                                         {}                     0.824005   
                                                       soup              {'n_neighbors': 7}     0.839763   
                                                                         {}                     0.814969   

                                                                                                       \
                                                                                                  std   
metric_name          classifier           dataset_name resampling_method clf_params                     
geometric_mean_score kneighborsclassifier new_ecoli    Not defined       {'n_neighbors': 7}  0.039082   
                                                                         {}                  0.025965   
                                                       globalcs          {'n_neighbors': 7}  0.032680   
                                                                         {}                  0.063314   
                                                       mdo               {'n_neighbors': 7}  0.040992   
                                                                         {}                  0.041023   
                                                       soup              {'n_neighbors': 7}  0.034917   
                                                                         {}                  0.059139   

                                                                                                       
                                                                                                  min  
metric_name          classifier           dataset_name resampling_method clf_params                    
geometric_mean_score kneighborsclassifier new_ecoli    Not defined       {'n_neighbors': 7}  0.805196  
                                                                         {}                  0.777511  
                                                       globalcs          {'n_neighbors': 7}  0.736093  
                                                                         {}                  0.724630  
                                                       mdo               {'n_neighbors': 7}  0.722135  
                                                                         {}                  0.755261  
                                                       soup              {'n_neighbors': 7}  0.789633  
                                                                         {}                  0.710178

Generate posthoc analysis for Wilcoxon test. You have to define names of classifiers, dataset names and metric names in query dict.

In [36]:
query_dict = {"classifier": ["decisiontreeclassifier", "kneighborsclassifier"], "metric_name": ["geometric_mean_score"], "dataset_name": ["glass", "new_ecoli"]}

results = AnalysisPipeline.generate_posthoc_analysis(query_dict, save_to_csv=False, csv_path=result_file.name ,posthoc_func_list=[[posthoc_wilcoxon, {}]])

In [37]:
results[0]

posthoc_wilcoxon_decisiontreeclassifier_geometric_mean_score_glass,globalcs,mdo,soup,Not defined
globalcs,1.000000,0.245487,0.058258,0.898317
mdo,0.245487,1.000000,0.621513,0.329983
soup,0.058258,0.621513,1.000000,0.044054
Not defined,0.898317,0.329983,0.044054,1.000000
